<a href="https://colab.research.google.com/github/infiniteoverflow/Estimator-Fashion-MNIST/blob/master/Estimator_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import tensorflow as tf
import numpy as np
print(tf.version)

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [21]:
LABEL_DIMENSIONS = 10

In [22]:
(train_images,train_labels) , (test_images,test_labels) = tf.keras.datasets.fashion_mnist.load_data()

In [23]:

print('train_images : {} train_labels : {}'.format(train_images.shape,train_labels.shape))
print('test_images : {} test_labels : {}'.format(test_images.shape,test_labels.shape))

train_images : (60000, 28, 28) train_labels : (60000,)
test_images : (10000, 28, 28) test_labels : (10000,)


In [24]:
TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)

In [25]:
TRAINING_SIZE , TEST_SIZE

(60000, 10000)

In [26]:
train_images = np.asarray(train_images,dtype=np.float32)/255
test_images = np.asarray(test_images,dtype=np.float32)/255

In [27]:
train_images = train_images.reshape((TRAINING_SIZE,28,28,1))
test_images = test_images.reshape((TEST_SIZE,28,28,1))

In [28]:
train_labels = tf.keras.utils.to_categorical(train_labels,num_classes=LABEL_DIMENSIONS)
test_labels = tf.keras.utils.to_categorical(test_labels,num_classes=LABEL_DIMENSIONS)

In [29]:
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

**Building a convolution model with `tf.keras` functional API**

In [30]:
inputs = tf.keras.Input(shape=(28,28,1))

x = tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu')(inputs)
x = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=2)(x)

x = tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=2)(x)

x = tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu')(x)


x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(64,activation='relu')(x)

predictions = tf.keras.layers.Dense(LABEL_DIMENSIONS,activation='softmax')(x)

In [31]:
model = tf.keras.Model(inputs=inputs,outputs=predictions)
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)              

**Compiling it**

In [32]:
optimizer = tf.keras.optimizers.SGD()
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

**Defining a strategy** 

In [33]:
strategy = tf.distribute.MirroredStrategy() # For CPU
config = tf.estimator.RunConfig(train_distribute=strategy)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not using Distribute Coordinator.


**Defining an estimator**

In [34]:
estimator = tf.keras.estimator.model_to_estimator(model,config=config)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp8mnpdonz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7fa344e51048>, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_distribute_coordina

**Input function for training and for testing**


In [35]:
def input_fn(images,labels,epochs,batch_size):
  dataset = tf.data.Dataset.from_tensor_slices((images,labels))
  SHUFFLE_SIZE = 5000
  dataset = dataset.shuffle(SHUFFLE_SIZE).repeat(epochs).batch(batch_size)
  dataset = dataset.prefetch(None)

  return dataset

**Training**

In [36]:
%time
BATCH_SIZE = 512
EPOCHS = 50

estimator_train_result = estimator.train(input_fn=lambda:input_fn(train_images,train_labels,EPOCHS,BATCH_SIZE))

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 9.06 µs
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Cause: could not parse the source code:

      lambda scaffold: scaffold.ready_op, args=(grouped_scaffold,))

This error may be avoided by creating the lambda in a standalone statement.

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code:

      lambda scaffold: scaffold.ready_op, args=(grouped_scaffold,))

This error may be avoided by creating the lambda in a standalone statement.

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/tmp/tmp8mnpdonz/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: /tmp/tmp8mnpdonz/keras/keras_model.ckpt
INFO

In [36]:
estimator_train_result.

In [37]:
estimator.evaluate(lambda:input_fn(test_images,test_labels,1,BATCH_SIZE))

INFO:tensorflow:Calling model_fn.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-09-02T12:29:00Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp8mnpdonz/model.ckpt-5860
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.40301s
INFO:tensorflow:Finished evaluation at 2020-09-02-12:29:00
INFO:tensorflow:Saving dict for global step 5860: accuracy = 0.8268, global_step = 5860, loss = 0.48587567
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5860: /tmp/tmp8mnpdonz/model.ckpt-5860


{'accuracy': 0.8268, 'global_step': 5860, 'loss': 0.48587567}